In [18]:
import torch 
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import numpy as np
import matplotlib.pyplot as plt
import csv 
import pandas as pd
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import StandardScaler


In [19]:
df = pd.read_csv("rc_dataset_2000.csv")
#print(df)
print(df.shape)

(2000, 6)


In [47]:


X = df[["R", "C", "Vin"]].values
Y = df[["V_out", "V_R", "V_C"]].values

scaler_x = StandardScaler()
scaler_y = StandardScaler()
X_tensor = torch.tensor(scaler_x.fit_transform(X), dtype = torch.float32)
Y_tensor = torch.tensor(scaler_y.fit_transform(Y), dtype = torch.float32)

print(X_tensor)
print(Y_tensor)

dataset = TensorDataset(X_tensor, Y_tensor)

loader = DataLoader(dataset, batch_size=32, shuffle=True)

tensor([[-0.8704,  0.1741, -1.6475],
        [-0.8704,  0.1741, -1.4741],
        [-0.8704,  0.1741, -1.3007],
        ...,
        [ 1.5667,  1.5667,  1.3007],
        [ 1.5667,  1.5667,  1.4741],
        [ 1.5667,  1.5667,  1.6475]])
tensor([[-1.6475, -0.1321, -1.6475],
        [-1.4741, -0.1321, -1.4741],
        [-1.3007, -0.1321, -1.3007],
        ...,
        [ 1.3007, -0.1321,  1.3007],
        [ 1.4741,  3.6499,  1.4741],
        [ 1.6475,  3.6499,  1.6475]])


In [48]:
class RegressionModel(nn.Module):
    def __init__(self, in_features, out_features):
        super().__init__()
        self.fc1 = nn.Linear(in_features, 3)
        self.fc2 = nn.Linear(3, 6)
        self.fc3 = nn.Linear(6, 9)
        self.fc4 = nn.Linear(9, 12)
        self.fc5 = nn.Linear(12, 12)
        self.fc6 = nn.Linear(12, 9)
        self.fc7 = nn.Linear(9, 6)
        self.fc8 = nn.Linear(6, 3)
        


    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        x = F.relu(self.fc6(x))
        x = F.relu(self.fc7(x))
        x = self.fc8(x)  # No activation on output for regression
        return x

In [49]:
in_features = 3
out_features = 3
model = RegressionModel(in_features, out_features)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr = 0.01)


In [50]:
nb_epochs = 50

for i in range(nb_epochs):
    for X_batch, Y_batch in loader:
        model.train()
        outputs = model(X_batch)
        loss = criterion(outputs, Y_batch)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Epoch [{i+ 1}], Loss: {loss.item():.9f}')

Epoch [1], Loss: 0.082562603
Epoch [2], Loss: 0.775973976
Epoch [3], Loss: 0.232294157
Epoch [4], Loss: 0.140482441
Epoch [5], Loss: 0.321235061
Epoch [6], Loss: 0.396956176
Epoch [7], Loss: 0.104675435
Epoch [8], Loss: 0.180279970
Epoch [9], Loss: 0.336393982
Epoch [10], Loss: 0.234092399
Epoch [11], Loss: 0.434014231
Epoch [12], Loss: 0.704813302
Epoch [13], Loss: 0.391313434
Epoch [14], Loss: 0.095412634
Epoch [15], Loss: 0.635142386
Epoch [16], Loss: 0.480720401
Epoch [17], Loss: 0.018517617
Epoch [18], Loss: 0.077687465
Epoch [19], Loss: 0.101870805
Epoch [20], Loss: 0.029790355
Epoch [21], Loss: 0.359979182
Epoch [22], Loss: 0.517319918
Epoch [23], Loss: 0.086974569
Epoch [24], Loss: 0.093557321
Epoch [25], Loss: 0.415019661
Epoch [26], Loss: 0.344673276
Epoch [27], Loss: 0.310204476
Epoch [28], Loss: 0.692131698
Epoch [29], Loss: 0.269258738
Epoch [30], Loss: 0.157022312
Epoch [31], Loss: 0.163435861
Epoch [32], Loss: 0.142388299
Epoch [33], Loss: 0.332230479
Epoch [34], Loss: 0

In [51]:
model.eval()

# No gradient calculation during evaluation
with torch.no_grad():
    # Forward pass
    predictions = model(X_tensor)  # Assuming X_tensor is your input data

    # Calculate the loss (optional)
    loss = criterion(predictions, Y_tensor)
    print(f"Evaluation Loss: {loss.item():.3f}")

    predictions_original = scaler_y.inverse_transform(predictions.numpy())

    # Print the inverse transformed predictions
    print("Inverse Predictions: \n", predictions_original)

Evaluation Loss: 0.242
Inverse Predictions: 
 [[ 1.24661729e-01 -2.01124234e-17  1.23355195e-01]
 [ 5.65551579e-01 -4.74017215e-18  5.64328492e-01]
 [ 1.04035974e+00  1.05326727e-17  1.03925061e+00]
 ...
 [ 8.34202290e+00  1.07026361e-15  8.34082317e+00]
 [ 9.26934719e+00  1.40407125e-15  9.26832390e+00]
 [ 9.92341900e+00  1.63951697e-15  9.92252064e+00]]
